# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.by import By

#abrindo navegador
navegador = webdriver.Chrome()

navegador.get('https://www.google.com/')

#dólar
navegador.find_element(
    "xpath", 
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação do dolar', Keys.ENTER)
cotacao_dolar = navegador.find_element(
    "xpath", 
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

5.1091999999999995


In [2]:
#euro
navegador = webdriver.Chrome()
navegador.get('https://www.google.com/')
navegador.find_element(
    "xpath", 
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotaçao euro', Keys.ENTER)
cotacao_euro = navegador.find_element(
    "xpath", 
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

5.5549777


In [3]:
#ouro
navegador = webdriver.Chrome()
navegador.get('https://www.melhorcambio.com/ouro-hoje#:~:text=O%20valor%20do%20grama%20do,%C3%A9%20de%20car%C3%A1ter%20exclusivamente%20informativo.')
cotacao_ouro = navegador.find_element("xpath", '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()


316.71


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [4]:
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)
print(tabela.info())

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Produtos         7 non-null      object 
 1   Preço Original   7 non-null      float64
 2   Moeda            7 non-null      object 
 3   Cotação          7 non-null      int64  
 4   Preço de Compra  7 non-null      float64
 5   Margem           7 non-null      float64
 6   Preço de Venda   7 non-null      float64
dtypes: float64(4), int64(1), object(2)
memory usage: 520.0+ bytes
None


- Atualizando os preços e o cálculo do Preço Final

In [5]:
#tabela ataualizada
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

#preço da compra = cotação * preço original
tabela['Preço de Compra'] = tabela['Cotação'] * tabela['Preço Original']

#preço de venda = preçp de compra * margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.109200,5109.148908,1.40,7152.808471
1,Carro Renault,4500.00,Euro,5.554978,24997.399650,2.00,49994.799300
2,Notebook Dell,899.99,Dólar,5.109200,4598.228908,1.70,7816.989144
3,IPhone,799.00,Dólar,5.109200,4082.250800,1.70,6939.826360
4,Carro Fiat,3000.00,Euro,5.554978,16664.933100,1.90,31663.372890
5,Celular Xiaomi,480.48,Dólar,5.109200,2454.868416,2.00,4909.736832
6,Joia 20g,20.00,Ouro,316.710000,6334.200000,1.15,7284.330000


### Agora vamos exportar a nova base de preços atualizada

In [6]:
tabela.to_excel("Produtos Novo.xlsx", index=False)